# Load dataset and seperate into train/test/validation

In [31]:
import tensorflow_datasets as tfds
import tensorflow as tf

batch_size = 32
training, validation, test = tfds.load('cats_vs_dogs', split=['train[:70%]', 'train[70%:80%]', 'train[90%:]'], as_supervised=True)

def preprocess(ds):
    ds = ds.cache()
    ds = ds.map(lambda x, y: (tf.image.rgb_to_grayscale(x), y))
    ds = ds.map(lambda x, y: (tf.image.resize(x, (256, 256)), y))
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

training = preprocess(training)
validation = preprocess(validation)
test = preprocess(test)

# Set up model

In [32]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['binary_accuracy'])

# Train model

In [33]:
epochs = 10
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/10
 84/509 [===>..........................] - ETA: 9:50 - loss: 0.6968 - binary_accuracy: 0.5108